In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2017-07-03 16:04:28
# @Author  : Lewis Tian (chasetichlewis@gmail.com)
# @Link    : https://github.com/LewisTian
# @Version : Python3.5

from google.colab import drive
from random import randint
import time
import sys
# from threading import Thread

global K

#FileName为读取文件名 (Nombre del Archivo leido)
def ReadData(FileName):
    data = open(FileName, 'r')
    line = data.readline()
    vertex = int(line.split(' ')[2])
    edge = int(line.split(' ')[3])
    print(vertex, int(edge))

    matrix = [([0] * vertex) for i in range(vertex)]
    while edge > 0:
        line = data.readline()
        a = line.split(' ')
        matrix[int(a[1]) - 1][int(a[2]) - 1] = 1
        matrix[int(a[2]) - 1][int(a[1]) - 1] = 1
        edge = edge - 1 
    data.close()
    return matrix

#计算相邻颜色表 (Calculando la tabla adyacente de color) ACT = n * k
def CountACT(ACT, matrix, color):
    n = len(matrix)
    for x in range(n):
        for y in range(x+1, n):
            if matrix[x][y]:
                ACT[x][color[y]] = ACT[x][color[y]] + 1
                ACT[y][color[x]] = ACT[y][color[x]] + 1
    return ACT

#计算评估函数f (calculo de la mfuncion evaluacion f:)
def CountF(ACT, color):
    f = 0
    n = len(color)
    for x in range(n):
        f = f + ACT[x][color[x]]
    return f

#选择一个邻域动作 (Eleccion de una accion - movimiento)
def OneMove(color, ACT, Tabu, iteration, f):
    move = [-1, 0, 0] #冲突减少量、顶点、新颜色 (Reduccion de conflictos, vector, new color )
    n = len(ACT) #顶点数 (numeros de vertices)
    m = len(ACT[1]) #颜色数 (numero de colores)
    bestCount = 2
    for x in range(n):
        i = ACT[x][color[x]] #当前节点的冲突数 (Numeros de conflictos en el nodo)
        if i:
            for y in range(m):
                if y != color[x]:
                    a = i - ACT[x][y]
                    if a > 0: #冲突减少>0 (reduccion de conflictos)
                        if a > move[0]:  #找出最优的一步
                            if Tabu[x][y] <= iteration: #没禁忌 (No taboo)
                                move = a, x, y
                                bestCount = 2
                            else: #禁忌 (taboo)
                                tmp_color = color[:]
                                tmp_color[x] = y
                                if  CountF(ACT, tmp_color) < f:
                                    move = a, x, y
                                    bestCount = 2
                        elif a == move[0] and Tabu[x][y] <= iteration: #若有多个最优则随机选择一个 (Si son muchos entonces elije uno aleatoriamente)
                            if randint(1, n) % bestCount == 0:
                                move = a, x, y
                            bestCount += 1
                    elif Tabu[x][y] <= iteration: #找不到能减少冲突的 (No puedo encontrar solucion para la reduccion)
                        if a > move[0]:
                            move = a, x, y
                            bestCount = 2
                        elif a == move[0]:
                            if randint(1, n) % bestCount == 0:  
                                move = a, x, y
                            bestCount += 1
    return move[1:] #顶点、新颜色 (vertice, new color)

def main(file, opt):
    k = K
    t1 = time.time()
    matrix = ReadData(file) #邻接矩阵 (Matriz de adyacencia)
    n = len(matrix) #顶点数 (Numero de vertice)
    color = [0 for i in range(n)]
    iteration = 0
    #初始化各顶点颜色(0 ~ k-1) (inicializacion de cada vector color)
    for i in range(n):
        color[i] = randint(0, k-1)
    while k >= opt:
        print("-------------------\n", k)
        Tabu = [([0] * k) for i in range(n)] #禁忌表 (Taboo List)
        ACT = [([0] * k) for i in range(n)] 
        ACT = CountACT(ACT, matrix, color) #相邻颜色表 (Tabla de color adyacente)
        f = CountF(ACT, color) #计算评估函数 (Calcula la funcion de evaluacion)
        bestF = f
        while f:
            test = 0
            move = OneMove(color, ACT, Tabu, iteration, bestF) #顶点，新颜色 (vertice, new color)
            oldColor = color[move[0]] #旧颜色 (viejo color)
            color[move[0]] = move[1] #新颜色 (nuevo color)
            #更新相邻颜色表 (actualizacion de los colores en la tabla adyacente)
            for x in range(n):
                if matrix[move[0]][x]:
                    ACT[x][oldColor] = ACT[x][oldColor] - 1
                    ACT[x][move[1]] = ACT[x][move[1]] + 1
            #禁忌 (Taboo)
            Tabu[move[0]][oldColor] = f + randint(1, 10) + iteration
            iteration = iteration + 1
            f = CountF(ACT, color)
            if f < bestF:
                bestF = f
                print(bestF,iteration)
            # print(f)
        k = k - 1
        for x in range(n):
            if color[x] == k:
                color[x] = randint(0, k-1)
    t2 = time.time()
    print("耗时(Tiempo de ejecucion): ", t2-t1)
    print(k, f, iteration)
    for x in range(n):
        for y in range(x+1, n):
            if matrix[x][y] and color[x] == color[y]:
                print(x, y, color[x], color[y])
    
if __name__ == '__main__':
    drive.mount('/content/gdrive')
    if len(sys.argv) < 3:
        print("文件名 初始颜色数 参考值(No se encuentra el archivo correcto)")
    else:
        file = "/content/gdrive/My Drive/DataSets/DSJC125_1.txt"
        K = 10
        val = 4
        main(file, val)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
125 736
-------------------
 10
128 1
118 2
110 3
102 4
94 5
86 6
80 7
76 8
72 9
68 10
64 11
60 12
56 13
52 14
48 15
44 16
40 17
36 18
32 19
30 20
28 21
26 22
24 23
22 24
20 25
18 26
16 27
14 28
12 29
10 30
8 31
6 32
4 33
2 34
0 35
-------------------
 9
34 36
26 37
20 38
14 39
10 40
6 41
4 42
2 43
0 44
-------------------
 8
54 45
46 46
40 47
34 48
28 49
24 50
22 51
20 52
18 53
16 54
14 55
12 56
10 57
8 58
6 59
4 60
2 61
0 63
-------------------
 7
42 64
34 65
28 66
24 67
22 68
20 69
18 70
16 71
14 72
12 73
10 79
8 81
6 85
4 87
2 89
0 92
-------------------
 6
46 93
40 94
36 95
32 96
28 97
26 98
24 99
22 100
20 101
18 102
16 103
14 104
12 106
10 113
8 116
6 118
4 123
2 128
0 175
-------------------
 5
86 176
80 177
74 178
70 179
66 180
62 181
58 182
54 183
52 184
50 185
48 186
46 187
44 188
42 189
40 190
38 191
36 192
34 193
32 194
30 197
28 198
26 207
24 